TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [38]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [39]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [40]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
#tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
tag_name = 'psychology'
#tag_name = 'science'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'
    
sequence_length = 40

In [41]:
with open(f'{root_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [42]:
corpus[0:500]

"So I want to start by offering you a free no-tech life hack, and all it requires of you is  that you change your posture for two minutes. But before I give it away, I want to ask you to right now do a little audit of your body and what you're doing with your body. So how many of you are sort of making yourselves smaller? Maybe you're hunching, crossing your legs, maybe wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out.  I see you. So I want you to pay atten"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [43]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—']

chars = [character for character in all_chars if character not in list_to_remove]
print(chars)
# hat tip to this post for the idea of removing one list from another
# https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

[' ', '!', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [44]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [45]:
# If running models with punctuation removed, leave commented out
#chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 65


In [46]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '.': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, '?': 13, 'A': 14, 'B': 15, 'C': 16, 'D': 17, 'E': 18, 'F': 19, 'G': 20, 'H': 21, 'I': 22, 'J': 23, 'K': 24, 'L': 25, 'M': 26, 'N': 27, 'O': 28, 'P': 29, 'Q': 30, 'R': 31, 'S': 32, 'T': 33, 'U': 34, 'V': 35, 'W': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


In [47]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '.', 3: '0', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9', 13: '?', 14: 'A', 15: 'B', 16: 'C', 17: 'D', 18: 'E', 19: 'F', 20: 'G', 21: 'H', 22: 'I', 23: 'J', 24: 'K', 25: 'L', 26: 'M', 27: 'N', 28: 'O', 29: 'P', 30: 'Q', 31: 'R', 32: 'S', 33: 'T', 34: 'U', 35: 'V', 36: 'W', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}


In [48]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 81,675 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [49]:
corpus[0:50]

'So I want to start by offering you a free notech l'

In [50]:
corpus[1:41]

'o I want to start by offering you a free'

In [51]:
corpus[41]

' '

In [52]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [53]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[32, 53, 0, 22, 0, 61, 39, 52, 58, 0, 58, 53, 0, 57, 58, 39, 56, 58, 0, 40, 63, 0, 53, 44, 44, 43, 56, 47, 52, 45, 0, 63, 53, 59, 0, 39, 0, 44, 56, 43]
[53, 0, 22, 0, 61, 39, 52, 58, 0, 58, 53, 0, 57, 58, 39, 56, 58, 0, 40, 63, 0, 53, 44, 44, 43, 56, 47, 52, 45, 0, 63, 53, 59, 0, 39, 0, 44, 56, 43, 43]


In [54]:
print ( y_numeric_list[0:2])

[43, 0]


In [55]:
indices_char[51]

'm'

In [56]:
corpus[0:43]

'So I want to start by offering you a free n'

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [57]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 81,635 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [58]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [59]:
X.shape

(81635, 40, 1)

In [60]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [61]:
# Look at the first value of X
#X[0]

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [62]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [63]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

If a model failed during fitting, we can still load it's most recent weights which were stored in a checkpoint file.

Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [64]:

checkpoint_dir = f'{root_path}{corpus_file_name}/'

latest = latest_checkpoint(checkpoint_dir)
latest

'../data/psychology_sentence_case/psychology_sentence_case.ckpt'

In [65]:
# # define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])


# #model = load_model(f'{latest}')

model = load_model(f'{root_path}models/ted_model_{tag_name}')

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 40, 400)           643200    
_________________________________________________________________
dropout_6 (Dropout)          (None, 40, 400)           0         
_________________________________________________________________
lstm_7 (LSTM)                (None, 128)               270848    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 65)                8385      
Total params: 922,433
Trainable params: 922,433
Non-trainable params: 0
_________________________________________________________________


X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [66]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 1517 sequence.
[53, 42, 63, 0, 50, 39, 52, 45, 59, 39, 45, 43, 0, 41, 53, 51, 51, 59, 52, 47, 41, 39, 58, 47, 52, 45, 0, 58, 53, 0, 51, 43, 13, 0, 36, 46, 39, 58, 57, 0]


Convert this back to text so we can see what it looks like.

In [67]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

'ody language communicating to me? Whats '

In [68]:
# We can also use the seed_start value to see a larger picture of the original text
for j in range(seed_start, seed_start + 15):
    seed_numeric = X_numeric_list[j]
    print (''.join([indices_char[letter_code] for letter_code in seed_numeric]))

ody language communicating to me? Whats 
dy language communicating to me? Whats m
y language communicating to me? Whats mi
 language communicating to me? Whats min
language communicating to me? Whats mine
anguage communicating to me? Whats mine 
nguage communicating to me? Whats mine c
guage communicating to me? Whats mine co
uage communicating to me? Whats mine com
age communicating to me? Whats mine comm
ge communicating to me? Whats mine commu
e communicating to me? Whats mine commun
 communicating to me? Whats mine communi
communicating to me? Whats mine communic
ommunicating to me? Whats mine communica


To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [69]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 65)
Prediction:
[[2.78809806e-03 4.22526441e-08 1.34004731e-05 4.03564741e-07
  3.60255683e-04 8.48298441e-06 2.15185602e-04 1.65061209e-08
  2.60885349e-06 1.41059099e-05 3.98016482e-07 1.82979605e-07
  6.71813323e-05 9.23226366e-07 5.36681873e-05 2.02538344e-04
  1.99578717e-05 8.52636062e-04 6.12997916e-04 2.67797277e-05
  1.82141830e-05 2.14508109e-04 3.19654047e-02 4.39010182e-05
  1.16947685e-05 2.08103975e-05 2.28580175e-04 6.52783419e-05
  2.03329000e-05 5.81788154e-05 3.33226815e-08 5.48093776e-05
  6.14478995e-05 6.03014705e-05 4.56137786e-06 3.51769245e-07
  1.06481066e-05 2.38711636e-05 1.13712273e-09 2.11722121e-01
  8.48094281e-03 4.44139168e-03 2.26547243e-03 3.18173468e-02
  4.48880903e-03 3.08582536e-03 1.36429546e-02 1.11214556e-01
  2.06961809e-03 1.48811971e-03 7.36126676e-03 2.75735203e-02
  2.59804726e-02 1.26010135e-01 7.03335460e-03 1.20797602e-03
  3.83334421e-03 2.67301667e-02 2.10821226e-01 1.78095866e-02
  4.65846155e-04 3.74667421e-

Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [70]:
np.argmax(prediction)

39

So the first letter predicted after the seed sequence is:

In [71]:
indices_char[np.argmax(prediction)]

'a'

In [72]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(500):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [73]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

['ody language communicating to me? Whats ->a sialat bor the wirh of the pomet whth the person to tell the person to tork',
 'of the person to to the perpon to tork of the posesful fxperience about the courre of stress response and soml bounect',
 'Thenes a little bot that and sower phople and workeng to be and that wour body language to to the people who were what',
 'they were wrrnd of she stress response and tome people that we are the like wes and we could be hi the seoe that wour',
 'heart  And then is what wou co they were walt to be willing to bo']

In [74]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

['ody language communicating to me? Whats mine communicating to you? And theres a lot of reason to believe that this is a',
 'valid way to look at this. So social scientists have spent a lot of time looking at the effects of our body language or',
 'other peoples body language on judgments. And we make sweeping judgments and inferences from body language. And those',
 'judgments can predict really meaningful life outcomes like who we hire or promote who we ask out on a date. For example',
 'Nalini Ambady a researcher at Tufts University shows that when people watch 30second soundless clips of real',
 'physicianpatient interactions their judgments of the physicians niceness predict whether or not that physician will be',
 'sued. So it doesnt have to do so much with whether or not that physician was incompetent but do we like that person and',
 'how they interacted? Even more dramatic Alex Todorov at Princeton has shown us that judgments of political candidates',
 'faces in just one seco